from network chuck 
https://www.youtube.com/watch?v=Wjrdr0NU4Sk&t=1236s

sudo apt update
sudo apt upgrade -y
curl -fsSL https://ollama.com/install.sh | sh


#put this in a browser. This is your endpoint.
localhost:11434

look for "ollama is running"



In [1]:
## pull and run your model - do this in terminal

ollama pull llama3.1
run llama3.1

#then ask a question in terminal

SyntaxError: invalid syntax (657957614.py, line 3)

In [3]:
wsl # to open a new window to profile your GPU
watch -n 0.5 nvidia-smi

⠋ ⠹ ⠸ ⠸ ⠼ ⠴ ⠦ ⠧ ⠏ ⠋ ⠋ ⠙ ⠸ ⠼ ⠴ ⠦ ⠧ ⠧ ⠇ ⠋ ⠋ >>> Send a message (/? for help)
Use Ctrl + d or /bye to exit.
>>> Send a message (/? for help)
>>> Send a message (/? for help)

In [ ]:
from there we install and run open-webui
need docker


#Install Docker
# Add Docker's official GPG key:
sudo apt-get update
sudo apt-get install ca-certificates curl
sudo install -m 0755 -d /etc/apt/keyrings
sudo curl -fsSL https://download.docker.com/linux/ubuntu/gpg -o /etc/apt/keyrings/docker.asc
sudo chmod a+r /etc/apt/keyrings/docker.asc

# Add the repository to Apt sources:
echo \
"deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu \
$(. /etc/os-release && echo "$VERSION_CODENAME") stable" | \
sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update

#Install Dockersudo 

sudo apt-get install docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin
     
#Run Open WebUi Docker Container
docker run -d --network=host -v open-webui:/app/backend/data -e OLLAMA_BASE_URL=http://127.0.0.1:11434 --name open-webui --restart always ghcr.io/open-webui/open-webui:main



### Reactors panel: Ollama endpoint
- Add a new reactor card titled **Ollama Endpoint** that mirrors the existing reactor shell (header, status badge, Start/Stop buttons, log console, and quick actions).
- Wire the **Start** button to run the install/start script shown below; it checks for `ollama` and installs it only when needed before calling `ollama serve` bound to `0.0.0.0:11434`.
- Keep the **Stop** button tied to the same process manager used by other reactors (e.g., `systemctl`, `supervisord`, or your `reactor_manager.py`) so status is always in sync.
- Surface health in the status badge by pinging `http://127.0.0.1:11434/api/tags` every few seconds; green when the request returns 200, amber while booting, red on failure.
- Reuse the log console region to stream the stdout/stderr from the helper process so operators can watch the install or serve output without leaving the panel.

In [ ]:
"""Helper routine for the Ollama reactor panel"""
import os
import shutil
import subprocess
import time
from pathlib import Path
from typing import Tuple
import requests
OLLAMA_INSTALL_SCRIPT = "https://ollama.com/install.sh"
OLLAMA_HOST = os.getenv("OLLAMA_HOST", "0.0.0.0")
OLLAMA_PORT = int(os.getenv("OLLAMA_PORT", 11434))
OLLAMA_ENDPOINT = f"http://127.0.0.1:{OLLAMA_PORT}"
def ensure_ollama() -> Tuple[bool, str]:
    """Download and install Ollama if it is missing."""
    if shutil.which("ollama"):
        return True, "ollama already installed"
    install_cmd = f"curl -fsSL {OLLAMA_INSTALL_SCRIPT} | sh"
    completed = subprocess.run(install_cmd, shell=True, text=True)
    if completed.returncode != 0:
        return False, "ollama install failed"
    return True, "ollama installed"
def start_ollama_daemon():
    """Start or reuse the ollama serve process bound to the configured host/port."""
    env = os.environ.copy()
    env["OLLAMA_HOST"] = OLLAMA_HOST
    env["OLLAMA_PORT"] = str(OLLAMA_PORT)
    return subprocess.Popen(["ollama", "serve"], env=env)
def wait_for_health(timeout: int = 30) -> bool:
    """Poll the tags endpoint until the service responds."""
    deadline = time.time() + timeout
    while time.time() < deadline:
        try:
            response = requests.get(f"{OLLAMA_ENDPOINT}/api/tags", timeout=2)
            if response.ok:
                return True
        except requests.RequestException:
            time.sleep(1)
    return False
if __name__ == "__main__":
    ok, msg = ensure_ollama()
    print(msg)
    if ok:
        process = start_ollama_daemon()
        print(f"ollama serve started with pid {process.pid}")
        if wait_for_health():
            print(f"✅ Ollama ready at {OLLAMA_ENDPOINT}")
        else:
            print("⚠️ Ollama did not pass health checks in time")

### Reactors panel: Interactive Ollama terminal window
- Create a sibling reactor window labeled **Ollama Terminal** that shares the same header, status display, and `Open Console`/`Close Console` controls already used in other terminal reactors.
- Route its iframe/webview to the WebSocket/terminal service your dashboard already hosts (e.g., `reactor_console.py`). Point the backend process at `python ollama_terminal.py` (script below) so the UI behaves exactly like the other terminal panes.
- When the main panel reports that the endpoint is healthy, automatically focus this terminal and seed it with a greeting message explaining the available commands (e.g., `/model`, `/clear`, `/quit`).
- Persist the last 50 prompts/responses so that reopening the terminal restores context for the on-call engineer. You can reuse the existing `reactor_logs` store that other panels use.

In [ ]:
"""Simple interactive console for the Ollama terminal reactor"""
import json
import os
import requests
OLLAMA_PORT = int(os.getenv("OLLAMA_PORT", 11434))
OLLAMA_CHAT_URL = f"http://127.0.0.1:{OLLAMA_PORT}/api/chat"
DEFAULT_MODEL = os.getenv("OLLAMA_MODEL", "llama3.1")
def stream_chat(prompt: str, model: str) -> str:
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "stream": False
    }
    response = requests.post(OLLAMA_CHAT_URL, json=payload, timeout=120)
    response.raise_for_status()
    data = response.json()
    return data.get("message", {}).get("content", "<no response>")
def run_terminal():
    print("Interactive Ollama terminal. Type /quit to exit, /model <name> to switch models.")
    model = DEFAULT_MODEL
    while True:
        user_input = input("You> ").strip()
        if not user_input:
            continue
        if user_input.lower() in {"/quit", "exit", "quit"}:
            print("Ending session.")
            break
        if user_input.startswith("/model"):
            parts = user_input.split(maxsplit=1)
            if len(parts) == 2:
                model = parts[1].strip() or model
                print(f"Model switched to {model}")
            else:
                print("Usage: /model <model_name>")
            continue
        if user_input == "/clear":
            os.system("clear")
            continue
        try:
            answer = stream_chat(user_input, model)
            print(f"Ollama> {answer}")
        except requests.RequestException as exc:
            print(f"⚠️ Request failed: {exc}")
if __name__ == "__main__":
    run_terminal()